In [1]:
import nest_asyncio
import requests
from multiprocessing.dummy import Pool
from lxml import etree
import json
import asyncio

In [2]:
nest_asyncio.apply()

In [3]:
async def down_videos(down_list):

    url = down_list[0] #url
    name = down_list[1] #name
    
    headers={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
    }
    print(f"正在下载视频 {name} ...") 
    video = requests.get(url=url, headers=headers).content #下载视频
    
    with open(f"./11_多线程_梨视频/{name}.mp4", "wb") as f: 
        f.write(video) 
        print(f"视频 {name} 下载完成！") 

In [4]:
down_url_list, down_list = [], []

url = 'https://www.pearvideo.com/' 

#获取首页的内容
headers={
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
}
response = requests.get(url=url, headers=headers)
res = etree.HTML(response.text)

#获取视频的ID
id_list = res.xpath('//a[@class="vervideo-lilink actplay"]/@href')
id_list = [id_list[i][6:] for i in range(3)]
#获取视频的json格式的url
url_list = [f'https://www.pearvideo.com/videoStatus.jsp?contId={id_list[i]}' for i in range(3)]
#获取视频名字
name_list = (res.xpath('//*[@class="vervideo-name"]/text()') + res.xpath('//*[@class="vervideo-title"]/text()'))[0:3]

#根据json格式获取视频的下载url
for i in range(len(url_list)):
    new_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36', 'Referer': f'https://www.pearvideo.com/{id_list[i]}'}
    response = requests.get(url=url_list[i], headers=new_headers).json() #获取json格式
    video_url = response["videoInfo"]["videos"]["srcUrl"] #获取下载url
    video_url = video_url.replace(video_url.split("/")[-1].split("-")[0], "cont-" + id_list[i])
    down_url_list.append(video_url)

#将下载url和视频名字写入列表中，方便多线程使用
for i in range(len(down_url_list)):
    down_list.append([down_url_list[i],name_list[i]])

In [5]:
tasks = []

for i in down_list:
    coroutine = down_videos(i)
    task = asyncio.ensure_future(coroutine)
    tasks.append(task)

asyncio.run(asyncio.wait(tasks))

正在下载视频 纪录片《我们记得你》:一个癌症艺术团的生命之歌 ...
视频 纪录片《我们记得你》:一个癌症艺术团的生命之歌 下载完成！
正在下载视频 宝藏老师｜小学老师课堂上现做爆米花，看馋一圈学生 ...
视频 宝藏老师｜小学老师课堂上现做爆米花，看馋一圈学生 下载完成！
正在下载视频 男子手脚萎缩嘴咬筷写诗开网店：发布上百首，3天咬碎一根筷 ...
视频 男子手脚萎缩嘴咬筷写诗开网店：发布上百首，3天咬碎一根筷 下载完成！


({<Task finished name='Task-1' coro=<down_videos() done, defined at <ipython-input-3-d2859130d5e8>:1> result=None>,
  <Task finished name='Task-2' coro=<down_videos() done, defined at <ipython-input-3-d2859130d5e8>:1> result=None>,
  <Task finished name='Task-3' coro=<down_videos() done, defined at <ipython-input-3-d2859130d5e8>:1> result=None>},
 set())